In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [2]:
import tensorflow as tf
from tensorflow import keras

# Locaation directory

In [3]:
train_dir = r'C:\Users\Suresh\cats_and_dogs_small\train'
validation_dir = r'C:\Users\Suresh\cats_and_dogs_small\validation'
test_dir = r'C:\Users\Suresh\cats_and_dogs_small\test'

# Data preprocessing

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
rotation_range = 40,
width_shift_range = 0.2,
height_shift_range = 0.2,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size = (150, 150),
batch_size = 20,
class_mode = "binary")

validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size = (150, 150),
batch_size = 20,
class_mode = "binary")


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Importing VGG16 & Creating model architecture

In [7]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (150, 150, 3))

58892288/58889256 [==============================] - 3s 0us/step


In [ ]:
conv_base.summary()

# Creating model

In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers
model= models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))

In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [19]:
#conv_base.trainable = False ( to reduce the trainable param of VGG16 and increase the training speed)

# Compiling the model

In [20]:
from tensorflow.keras import optimizers

model.compile(loss = "binary_crossentropy",
             optimizer = optimizers.RMSprop(lr = 2e-5),
             metrics = ["acc"])

# Model training

In [25]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("CNN_Project_Model-{epoch:02d}.h5", save_best_only = True)

In [26]:
history = model.fit_generator(
train_generator,
steps_per_epoch = 100,
epochs = 30,
validation_data = validation_generator,
validation_steps = 50,
callbacks = [checkpoint_cb])

Epoch 1/30
100/100 [==============================] - 707s 7s/step - loss: 0.2729 - acc: 0.8855 - val_loss: 0.1334 - val_acc: 0.9440
Epoch 2/30
 14/100 [===>..........................] - ETA: 8:38 - loss: 0.2531 - acc: 0.8821

KeyboardInterrupt: 

# Graph

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

# Testing the model

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size = (150, 150),
batch_size = 20,
class_mode = "binary")

In [ ]:
model.evaluate_generator(test_generator, steps = 50)